#### Visualization of Xenium add-on panel healthy gut data after the labels transer 
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 20th March 2024

##### Import packages

In [ ]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix

In [ ]:
import tifffile as tf
from PIL import Image, ImageFile

#### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

#### Data upload

+ Load h5ad object

In [ ]:
adata_xenium = sc.read_10x_h5('/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5')

+ Load image and convert it from tiff to png

In [ ]:
ome_tiff_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif'

In [ ]:
# Replace 'path/to/your/output_image.png' with the path where you want to save the PNG image
png_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.png'

# Read the OME-TIFF image
img = tf.imread(ome_tiff_path)

# Convert the image to an array and ensure it's in 8-bit format if it's not already
if img.dtype != np.uint8:
    # Scale to 0-255 if it's not in uint8 format
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

# Convert to PIL Image and save as PNG
Image.fromarray(img).save(png_path)

print(f"Converted {ome_tiff_path} to {png_path}")

+ Rotate image

In [ ]:
Image.MAX_IMAGE_PIXELS = None

# Path to your original PNG image
original_image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.png'
# Path where the rotated image will be saved
rotated_image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_rotated.png'

# Open the image
original_image = Image.open(original_image_path)

# Rotate the image by 90 degrees
rotated_image = original_image.rotate(90, expand=True)

# Save the rotated image
rotated_image.save(rotated_image_path)

print(f"Rotated image saved as {rotated_image_path}")

+ Upload image

In [ ]:
Image.MAX_IMAGE_PIXELS = None
hires = np.asarray(Image.open('/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Xenium_rotated_image.png'))

+ Upload coordinates

In [ ]:
coords = pd.read_csv(
        "/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cells.csv",
        header=0)

In [ ]:
# rename index column in adata_xenium to 'cell_id'
adata_xenium.obs.index.name = 'cell_id'

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(coords, on='cell_id', how='left')

In [ ]:
adata_xenium.obsm["spatial"] = adata_xenium.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

In [ ]:
nucleus_boundaries = pd.read_csv(
        "/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/nucleus_boundaries.csv",
        header=0)

In [ ]:
adata_xenium

In [ ]:
spatial_key = "spatial"
library_id = "tissue42"
adata_xenium.uns[spatial_key] = {library_id: {}}
adata_xenium.uns[spatial_key][library_id]["images"] = {}
adata_xenium.uns[spatial_key][library_id]["images"] = {"hires": hires}

In [ ]:
adata_xenium.uns

In [ ]:
# check class of adata_xenium.uns
print(type(adata_xenium.uns['spatial']['tissue42']['images']['hires']))

#### Visualise predicted cell types

+ Upload the object with cell types

In [ ]:
adata_full = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/scRNA_and_Xenium_add_on_data_scANVI.h5ad')

In [ ]:
adata_full.obs.index.name = 'cell_id'

In [ ]:
adata_xenium.obs.set_index('cell_id', inplace=True)

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(adata_full.obs[['C_scANVI']], on='cell_id', how='left')

In [ ]:
sq.pl.spatial_scatter(adata_xenium)

In [ ]:
adata_xenium.uns['spatial']['tissue42']['images']['hires']

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='hires',
    shape=None,
    color=[
        "C_scANVI",
    ],
    size=0.1,
    alpha=1
)

+ Add epithelial labels

In [ ]:
adata_epithelial_labels = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/epithelial_scRNA_and_Xenium_add_on_scANVI.h5ad')

In [ ]:
adata_epithelial_labels = adata_epithelial_labels[adata_epithelial_labels.obs['Study_name'] == '10X_Xenium_gut_data']

In [ ]:
# transfer the epithelial labels to the adata_xenium object in Cell_States column, rest fill with 'Unknown'
adata_xenium.obs['Cell_States'] = 'Unknown'
adata_xenium.obs.loc[adata_epithelial_labels.obs.index, 'Cell_States'] = adata_epithelial_labels.obs['C_scANVI']

In [ ]:
adata_xenium.obs['Cell_States'].value_counts()  

+ Add stem cells labels

In [ ]:
adata_stem_cells = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Stem_cells_scRNA_and_Xenium_add_on_scANVI.h5ad')

In [ ]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs['Study_name'] == '10X_Xenium_gut_data']

In [ ]:
adata_xenium.obs['Stem_cells'] = 'Unknown'
adata_xenium.obs.loc[adata_stem_cells.obs.index, 'Stem_cells'] = adata_stem_cells.obs['C_scANVI']

+ Write anndata object

In [ ]:
adata_xenium.write('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/10X_Xenium_add-on_with_cell_states_and_image.h5ad')

+ Visualize

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='hires',
    shape=None,
    color=[
        "Cell_States",
    ],
    size=0.1,
    alpha=1
)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='hires',
    shape=None,
    color=[
        "Stem_cells",
    ],
    size=0.1,
    alpha=1
)